In [77]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

In [78]:
# objective function
def objective(x):
	return x[0]**2.0 + x[1]**2.0

In [79]:
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded

In [80]:
# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better 
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

In [81]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [82]:
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

In [83]:
# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	# enumerate generations
	for gen in range(n_iter):
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d) for d in decoded]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

# define range for input
bounds = [[1, 12], [1, 12]]
# define the total iterations
n_iter = 3300
# bits per variable
n_bits = 12
# define the population size
n_pop = 1960
# crossover rate
r_cross = 1
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>0, new best f([2060.4345703125, 1846.58203125]) = 7655255.816674
>1, new best f([2101.0009765625, 1743.85986328125]) = 7455252.326280
>1, new best f([2007.4365234375, 1750.30517578125]) = 7093369.603997
>2, new best f([1965.5615234375, 1724.12109375]) = 6836025.648332
>4, new best f([1975.3759765625, 1695.52001953125]) = 6776898.385412
>4, new best f([1962.9443359375, 1698.74267578125]) = 6738877.144510
>5, new best f([1963.271484375, 1695.52001953125]) = 6729223.057991
>6, new best f([1962.9443359375, 1669.73876953125]) = 6641178.024465
>9, new best f([1960.3271484375, 1672.15576171875]) = 6638987.420350
>10, new best f([1960.3271484375, 1671.7529296875]) = 6637640.386820
>11, new best f([1960.3271484375, 1670.947265625]) = 6634947.293401
>13, new best f([1960.3271484375, 1658.8623046875]) = 6594706.674814
>15, new best f([1960.3271484375, 1658.056640625]) = 6592034.352422
>16, new best f([1960.0, 1658.056640625]) = 6590751.823521
>16, new best f([1960.3271484375, 1651.611328125]) = 